# Bitcoin Stock price Prediction using LSTM

![](https://media0.giphy.com/media/f67U9Xc53i4ViUs5T2/giphy.gif?cid=ecf05e47h4dpv8s5ppc6omcbb5uzwprey8y97x3fy8qk8dk8&rid=giphy.gif&ct=g) 

<iframe width="1024" height="576" src="https://www.youtube.com/embed/p-QY7JNGD60" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

# [](http://) Table of Content 
<hr style='height:2px'>

## 1. What is LSTM?
## 2. Importing Library
## 3. Loading Dataset
## 4. EDA
## 5. Biulding Model
## 6. Prediction
## 7. Evaluation 
## 8. Conclusion
<hr style='height:2px'>

# 1. What is LSTM ?

* ###  Long short-term memory is an artificial recurrent neural network architecture used in the field of deep learning. Unlike standard feedforward neural networks, LSTM has feedback connections. It can process not only single data points, but also entire sequences of data.

* ### Long Short-Term Memory (LSTM) networks are a type of recurrent neural network capable of learning order dependence in sequence prediction problems. This is a behavior required in complex problem domains like machine translation, speech recognition, and more. LSTMs are a complex area of deep learning.

* ### LSTMs are often referred to as fancy RNNs. Vanilla RNNs do not have a cell state. They only have hidden states and those hidden states serve as the memory for RNNs. Meanwhile, LSTM has both cell states and a hidden states.

![image](https://blog.floydhub.com/content/images/2019/06/Slide19.JPG)

# 2. Importing Library

In [1]:
# First we will import the necessary Library 

import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# For Evalution we will use these library

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# For model building we will use these library

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM


# For PLotting we will use these library

import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 3. Loading Dataset

#  [We can use this link to download bitcoin dataset from yahoo finance](https://finance.yahoo.com/quote/BTC-USD/history?p=BTC-USD)

In [2]:
# Load our dataset 
# Note it should be in same dir


maindf=pd.read_csv('/content/BTC-USD.csv')

In [3]:
print('Total number of days present in the dataset: ',maindf.shape[0])
print('Total number of fields present in the dataset: ',maindf.shape[1])

Total number of days present in the dataset:  2713
Total number of fields present in the dataset:  7


In [4]:
maindf.shape

(2713, 7)

In [5]:
maindf.head()

Date        Open        High         Low       Close   Adj Close  \
0  2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
1  2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2  2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
3  2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
4  2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

     Volume  
0  21056800  
1  34483200  
2  37919700  
3  36863600  
4  26580100

In [6]:
maindf.tail()

Date          Open          High           Low         Close  \
2708  2022-02-15  42586.464844  44667.218750  42491.035156  44575.203125   
2709  2022-02-16  44578.277344  44578.277344  43456.691406  43961.859375   
2710  2022-02-17  43937.070313  44132.972656  40249.371094  40538.011719   
2711  2022-02-18  40552.132813  40929.152344  39637.617188  40030.976563   
2712  2022-02-19  40022.132813  40246.027344  40010.867188  40126.429688   

         Adj Close       Volume  
2708  44575.203125  22721659051  
2709  43961.859375  19792547657  
2710  40538.011719  26246662813  
2711  40030.976563  23310007704  
2712  40126.429688  22263900160

In [7]:
maindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2713 non-null   object 
 1   Open       2713 non-null   float64
 2   High       2713 non-null   float64
 3   Low        2713 non-null   float64
 4   Close      2713 non-null   float64
 5   Adj Close  2713 non-null   float64
 6   Volume     2713 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 148.5+ KB


In [8]:
maindf.describe()

Open          High           Low         Close     Adj Close  \
count   2713.000000   2713.000000   2713.000000   2713.000000   2713.000000   
mean   11311.041069  11614.292482  10975.555057  11323.914637  11323.914637   
std    16106.428891  16537.390649  15608.572560  16110.365010  16110.365010   
min      176.897003    211.731003    171.509995    178.102997    178.102997   
25%      606.396973    609.260986    604.109985    606.718994    606.718994   
50%     6301.569824   6434.617676   6214.220215   6317.609863   6317.609863   
75%    10452.399414  10762.644531  10202.387695  10462.259766  10462.259766   
max    67549.734375  68789.625000  66382.062500  67566.828125  67566.828125   

             Volume  
count  2.713000e+03  
mean   1.470462e+10  
std    2.001627e+10  
min    5.914570e+06  
25%    7.991080e+07  
50%    5.098183e+09  
75%    2.456992e+10  
max    3.509679e+11

# Checking for Null Values

In [9]:
print('Null Values:',maindf.isnull().values.sum())

Null Values: 0


In [10]:
print('NA values:',maindf.isnull().values.any())

NA values: False


In [11]:
# If dataset had null values we can use this code to drop all the null values present in the dataset

# maindf=maindf.dropna()
# print('Null Values:',maindf.isnull().values.sum())
# print('NA values:',maindf.isnull().values.any())

In [12]:
# Final shape of the dataset after dealing with null values 

maindf.shape

(2713, 7)

# 4. EDA(Exploratory Data Analysis)

In [13]:
# Printing the start date and End date of the dataset

sd=maindf.iloc[0][0]
ed=maindf.iloc[-1][0]


print('Starting Date',sd)
print('Ending Date',ed)


Starting Date 2014-09-17
Ending Date 2022-02-19


#### StockPrice Analysis from Start

In [14]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2014 = maindf.loc[(maindf['Date'] >= '2014-09-17')
                     & (maindf['Date'] < '2014-12-31')]

y_2014.drop(y_2014[['Adj Close','Volume']],axis=1)

Date        Open        High         Low       Close
0   2014-09-17  465.864014  468.174011  452.421997  457.334015
1   2014-09-18  456.859985  456.859985  413.104004  424.440002
2   2014-09-19  424.102997  427.834991  384.532013  394.795990
3   2014-09-20  394.673004  423.295990  389.882996  408.903992
4   2014-09-21  408.084991  412.425995  393.181000  398.821014
..         ...         ...         ...         ...         ...
100 2014-12-26  319.152008  331.424011  316.627014  327.924011
101 2014-12-27  327.583008  328.911011  312.630005  315.863007
102 2014-12-28  316.160004  320.028015  311.078003  317.239014
103 2014-12-29  317.700989  320.266998  312.307007  312.670013
104 2014-12-30  312.718994  314.808990  309.372986  310.737000

[105 rows x 5 columns]

In [15]:
monthvise= y_2014.groupby(y_2014['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open       Close
Date                             
January           NaN         NaN
February          NaN         NaN
March             NaN         NaN
April             NaN         NaN
May               NaN         NaN
June              NaN         NaN
July              NaN         NaN
August            NaN         NaN
September  412.654003  407.182428
October    365.748000  364.148873
November   364.850235  366.099799
December   344.146864  341.970366

 Note that we only have few months in 2014 so the rest of the months are **NaN**

In [16]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

Note that we only have few months in 2014 so the rest of the months are not plotted since we do not have the data

In [17]:
y_2014.groupby(y_2014['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2014.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2014.groupby(y_2014['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

Note that we only have few months in 2014 so the rest of the months are not plotted since we do not have the data

In [18]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2014, x=y_2014.Date, y=[y_2014['Open'], y_2014['Close'], 
                                          y_2014['High'], y_2014['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2015

In [19]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2015 = maindf.loc[(maindf['Date'] >= '2015-01-01')
                     & (maindf['Date'] < '2016-01-01')]

y_2015.drop(y_2015[['Adj Close','Volume']],axis=1)

Date        Open        High         Low       Close
106 2015-01-01  320.434998  320.434998  314.002991  314.248993
107 2015-01-02  314.079010  315.838989  313.565002  315.032013
108 2015-01-03  314.846008  315.149994  281.082001  281.082001
109 2015-01-04  281.145996  287.230011  257.612000  264.195007
110 2015-01-05  265.084015  278.341003  265.084015  274.473999
..         ...         ...         ...         ...         ...
466 2015-12-27  416.514008  424.006989  408.882996  422.822998
467 2015-12-28  423.342987  429.769012  418.480988  422.278992
468 2015-12-29  422.097992  432.983002  420.627014  432.983002
469 2015-12-30  433.299988  434.386993  422.084015  426.619995
470 2015-12-31  425.875000  432.920990  418.734985  430.566986

[365 rows x 5 columns]

In [20]:
monthvise= y_2015.groupby(y_2015['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open       Close
Date                             
January    251.799905  248.782547
February   232.821856  234.153645
March      269.278419  269.042259
April      235.708667  235.491534
May        237.161806  236.997001
June       236.941433  238.081766
July       278.857679  279.563740
August     252.986774  250.733805
September  233.486733  233.595533
October    262.306000  264.855356
November   346.866833  348.883332
December   422.618033  424.464547

In [21]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [22]:
y_2015.groupby(y_2015['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2015.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2015.groupby(y_2015['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [23]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2015, x=y_2015.Date, y=[y_2015['Open'], y_2015['Close'], 
                                          y_2015['High'], y_2015['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2016

In [24]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2016 = maindf.loc[(maindf['Date'] >= '2016-01-01')
                     & (maindf['Date'] < '2017-01-01')]

y_2016.drop(y_2016[['Adj Close','Volume']],axis=1)

Date        Open        High         Low       Close
471 2016-01-01  430.721008  436.246002  427.515015  434.334015
472 2016-01-02  434.622009  436.062012  431.869995  433.437988
473 2016-01-03  433.578003  433.743011  424.705994  430.010986
474 2016-01-04  430.061005  434.516998  429.084015  433.091003
475 2016-01-05  433.069000  434.182007  429.675995  431.959991
..         ...         ...         ...         ...         ...
832 2016-12-27  908.354004  940.047974  904.255005  933.197998
833 2016-12-28  934.830994  975.921021  934.830994  975.921021
834 2016-12-29  975.125000  979.396973  954.502991  973.497009
835 2016-12-30  972.534973  972.534973  934.833008  961.237976
836 2016-12-31  960.627014  963.742981  947.236023  963.742981

[366 rows x 5 columns]

In [25]:
monthvise= y_2016.groupby(y_2016['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open       Close
Date                             
January    412.805902  410.844485
February   402.304692  404.408274
March      417.262033  416.525774
April      433.487433  434.339398
May        459.237547  461.954415
June       638.544834  642.869061
July       662.977779  661.356103
August     581.238966  579.585197
September  604.614034  605.848633
October    640.702546  643.550935
November   725.073804  726.349101
December   821.108255  828.060356

In [26]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [27]:
y_2016.groupby(y_2016['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2016.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2016.groupby(y_2016['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [28]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2016, x=y_2016.Date, y=[y_2016['Open'], y_2016['Close'], 
                                          y_2016['High'], y_2016['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2017

In [29]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2017 = maindf.loc[(maindf['Date'] >= '2017-01-01')
                     & (maindf['Date'] < '2018-01-01')]

y_2017.drop(y_2017[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
837  2017-01-01    963.658020   1003.080017    958.698975    998.325012
838  2017-01-02    998.617004   1031.390015    996.702026   1021.750000
839  2017-01-03   1021.599976   1044.079956   1021.599976   1043.839966
840  2017-01-04   1044.400024   1159.420044   1044.400024   1154.729980
841  2017-01-05   1156.729980   1191.099976    910.416992   1013.380005
...         ...           ...           ...           ...           ...
1197 2017-12-27  16163.500000  16930.900391  15114.299805  15838.500000
1198 2017-12-28  15864.099609  15888.400391  13937.299805  14606.500000
1199 2017-12-29  14695.799805  15279.000000  14307.000000  14656.200195
1200 2017-12-30  14681.900391  14681.900391  12350.099609  12952.200195
1201 2017-12-31  12897.700195  14377.400391  12755.599609  14156.400391

[365 rows x 5 columns]

In [30]:
monthvise= y_2017.groupby(y_2017['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January      914.680971    914.916159
February    1055.620071   1062.533672
March       1133.212576   1129.365228
April       1197.646997   1206.641007
May         1865.748712   1895.383529
June        2630.573332   2636.204346
July        2509.213233   2519.418386
August      3819.812579   3880.989998
September   4077.400993   4064.836312
October     5291.370007   5360.071604
November    7685.745996   7813.132975
December   15175.219601  15294.270980

In [31]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [32]:
y_2017.groupby(y_2017['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2017.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2017.groupby(y_2017['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [33]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2017, x=y_2017.Date, y=[y_2017['Open'], y_2017['Close'], 
                                          y_2017['High'], y_2017['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2018

In [34]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2018 = maindf.loc[(maindf['Date'] >= '2018-01-01')
                     & (maindf['Date'] < '2019-01-01')]

y_2018.drop(y_2018[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
1202 2018-01-01  14112.200195  14112.200195  13154.700195  13657.200195
1203 2018-01-02  13625.000000  15444.599609  13163.599609  14982.099609
1204 2018-01-03  14978.200195  15572.799805  14844.500000  15201.000000
1205 2018-01-04  15270.700195  15739.700195  14522.200195  15599.200195
1206 2018-01-05  15477.200195  17705.199219  15202.799805  17429.500000
...         ...           ...           ...           ...           ...
1562 2018-12-27   3854.688477   3874.416992   3645.448486   3654.833496
1563 2018-12-28   3653.131836   3956.135986   3642.632080   3923.918701
1564 2018-12-29   3932.491699   3963.758789   3820.408691   3820.408691
1565 2018-12-30   3822.384766   3901.908936   3797.219238   3865.952637
1566 2018-12-31   3866.839111   3868.742920   3725.867432   3742.700439

[365 rows x 5 columns]

In [35]:
monthvise= y_2018.groupby(y_2018['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January    13212.074219  13085.558090
February    9462.242920   9472.001151
March       9156.591718   9040.557097
April       7963.618311   8033.596631
May         8505.240675   8450.997732
June        6829.257975   6793.507666
July        7101.466450   7146.349987
August      6723.800955   6700.129946
September   6622.821338   6610.675033
October     6494.016491   6485.118747
November    5481.615120   5404.250171
December    3726.475106   3717.488344

In [36]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [37]:
y_2018.groupby(y_2018['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2018.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2018.groupby(y_2018['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [38]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2018, x=y_2018.Date, y=[y_2018['Open'], y_2018['Close'], 
                                          y_2018['High'], y_2018['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2019

In [39]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2019 = maindf.loc[(maindf['Date'] >= '2019-01-01')
                     & (maindf['Date'] < '2020-01-01')]

y_2019.drop(y_2019[['Adj Close','Volume']],axis=1)

Date         Open         High          Low        Close
1567 2019-01-01  3746.713379  3850.913818  3707.231201  3843.520020
1568 2019-01-02  3849.216309  3947.981201  3817.409424  3943.409424
1569 2019-01-03  3931.048584  3935.685059  3826.222900  3836.741211
1570 2019-01-04  3832.040039  3865.934570  3783.853760  3857.717529
1571 2019-01-05  3851.973877  3904.903076  3836.900146  3845.194580
...         ...          ...          ...          ...          ...
1927 2019-12-27  7238.141113  7363.529297  7189.934082  7290.088379
1928 2019-12-28  7289.031250  7399.041016  7286.905273  7317.990234
1929 2019-12-29  7317.647461  7513.948242  7279.865234  7422.652832
1930 2019-12-30  7420.272949  7454.824219  7276.308105  7292.995117
1931 2019-12-31  7294.438965  7335.290039  7169.777832  7193.599121

[365 rows x 5 columns]

In [40]:
monthvise= y_2019.groupby(y_2019['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January     3709.705645   3701.554963
February    3697.178327   3711.907261
March       3967.740400   3976.069100
April       5136.813314   5178.469434
May         7205.208024   7309.694131
June        9339.480322   9415.900179
July       10691.706055  10669.336158
August     10657.745621  10643.248362
September   9858.141813   9814.067871
October     8382.432129   8411.929168
November    8427.103516   8373.572412
December    7296.351625   7284.013042

In [41]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [42]:
y_2019.groupby(y_2019['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2019.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2019.groupby(y_2019['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [43]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2019, x=y_2019.Date, y=[y_2019['Open'], y_2019['Close'], 
                                          y_2019['High'], y_2019['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2020

In [44]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2020 = maindf.loc[(maindf['Date'] >= '2020-01-01')
                     & (maindf['Date'] < '2021-01-01')]

y_2020.drop(y_2020[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
1932 2020-01-01   7194.892090   7254.330566   7174.944336   7200.174316
1933 2020-01-02   7202.551270   7212.155273   6935.270020   6985.470215
1934 2020-01-03   6984.428711   7413.715332   6914.996094   7344.884277
1935 2020-01-04   7345.375488   7427.385742   7309.514160   7410.656738
1936 2020-01-05   7410.451660   7544.497070   7400.535645   7411.317383
...         ...           ...           ...           ...           ...
2293 2020-12-27  26439.373047  28288.839844  25922.769531  26272.294922
2294 2020-12-28  26280.822266  27389.111328  26207.640625  27084.808594
2295 2020-12-29  27081.810547  27370.720703  25987.298828  27362.437500
2296 2020-12-30  27360.089844  28937.740234  27360.089844  28840.953125
2297 2020-12-31  28841.574219  29244.876953  28201.992188  29001.720703

[366 rows x 5 columns]

In [45]:
monthvise= y_2020.groupby(y_2020['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January     8318.949597   8389.270476
February    9656.215113   9630.722185
March       6943.507009   6871.016113
April       7150.611328   7224.477328
May         9237.761530   9263.151745
June        9499.797005   9489.227214
July        9519.383852   9589.899729
August     11639.097215  11652.394185
September  10689.700163  10660.276856
October    11791.307491  11886.978201
November   16450.121647  16645.757422
December   21680.540827  21983.137097

In [46]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [47]:
y_2020.groupby(y_2020['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2020.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2020.groupby(y_2020['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [48]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2020, x=y_2020.Date, y=[y_2020['Open'], y_2020['Close'], 
                                          y_2020['High'], y_2020['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Analysis of Year 2021

In [49]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2021 = maindf.loc[(maindf['Date'] >= '2021-01-01')
                     & (maindf['Date'] < '2021-12-31')]

y_2021.drop(y_2021[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
2298 2021-01-01  28994.009766  29600.626953  28803.585938  29374.152344
2299 2021-01-02  29376.455078  33155.117188  29091.181641  32127.267578
2300 2021-01-03  32129.408203  34608.558594  32052.316406  32782.023438
2301 2021-01-04  32810.949219  33440.218750  28722.755859  31971.914063
2302 2021-01-05  31977.041016  34437.589844  30221.187500  33992.429688
...         ...           ...           ...           ...           ...
2657 2021-12-26  50428.691406  51196.378906  49623.105469  50809.515625
2658 2021-12-27  50802.609375  51956.328125  50499.468750  50640.417969
2659 2021-12-28  50679.859375  50679.859375  47414.210938  47588.855469
2660 2021-12-29  47623.871094  48119.742188  46201.496094  46444.710938
2661 2021-12-30  46490.605469  47879.964844  46060.312500  47178.125000

[364 rows x 5 columns]

In [50]:
monthvise= y_2021.groupby(y_2021['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January    34652.961694  34761.649950
February   45874.967216  46306.798968
March      54544.678176  54998.008695
April      57251.256250  57206.720052
May        47105.828503  46443.286668
June       35920.546940  35845.154688
July       34234.212450  34444.973790
August     45516.119834  45709.022682
September  46041.859375  45939.771484
October    57344.743952  57911.970514
November   60857.520313  60621.488802
December   49753.779818  49361.767969

#### Since we had data till 24-08-2021 in Months after August its showing NaN

In [51]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [52]:
y_2021.groupby(y_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2021.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2021.groupby(y_2021['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [53]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2021, x=y_2021.Date, y=[y_2021['Open'], y_2021['Close'], 
                                          y_2021['High'], y_2021['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

In [54]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_2022 = maindf.loc[(maindf['Date'] >= '2022-01-01')
                     & (maindf['Date'] < '2022-12-31')]

y_2022.drop(y_2015[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
2663 2022-01-01  46311.746094  47827.312500  46288.484375  47686.812500
2664 2022-01-02  47680.925781  47881.406250  46856.937500  47345.218750
2665 2022-01-03  47343.542969  47510.726563  45835.964844  46458.117188
2666 2022-01-04  46458.851563  47406.546875  45752.464844  45897.574219
2667 2022-01-05  45899.359375  46929.046875  42798.222656  43569.003906
2668 2022-01-06  43565.511719  43748.718750  42645.539063  43160.929688
2669 2022-01-07  43153.570313  43153.570313  41077.445313  41557.902344
2670 2022-01-08  41561.464844  42228.941406  40672.277344  41733.941406
2671 2022-01-09  41734.726563  42663.949219  41338.160156  41911.601563
2672 2022-01-10  41910.230469  42199.484375  39796.570313  41821.261719
2673 2022-01-11  41819.507813  43001.156250  41407.753906  42735.855469
2674 2022-01-12  42742.179688  44135.367188  42528.988281  43949.101563
2675 2022-01-13  43946.742188  44278.421875  42447.042969  42591.570313
2676 2022-01-14  42598.871094  43346.687500  41982.617188  43099.699219
2677 2022-01-15  43101.898438  43724.671875  42669.035156  43177.398438
2678 2022-01-16  43172.039063  43436.808594  42691.023438  43113.878906
2679 2022-01-17  43118.121094  43179.390625  41680.320313  42250.550781
2680 2022-01-18  42250.074219  42534.402344  41392.214844  42375.632813
2681 2022-01-19  42374.039063  42478.304688  41242.914063  41744.328125
2682 2022-01-20  41744.027344  43413.023438  40672.824219  40680.417969
2683 2022-01-21  40699.605469  41060.527344  35791.425781  36457.316406
2684 2022-01-22  36471.589844  36688.812500  34349.250000  35030.250000
2685 2022-01-23  35047.359375  36433.312500  34784.968750  36276.804688
2686 2022-01-24  36275.734375  37247.519531  33184.058594  36654.328125
2687 2022-01-25  36654.804688  37444.570313  35779.429688  36954.003906
2688 2022-01-26  36950.515625  38825.410156  36374.906250  36852.121094
2689 2022-01-27  36841.878906  37148.324219  35629.281250  37138.234375
2690 2022-01-28  37128.445313  37952.878906  36211.109375  37784.332031
2691 2022-01-29  37780.714844  38576.261719  37406.472656  38138.179688
2692 2022-01-30  38151.917969  38266.339844  37437.710938  37917.601563
2693 2022-01-31  37920.281250  38647.261719  36733.574219  38483.125000
2694 2022-02-01  38481.765625  39115.132813  38113.664063  38743.273438
2695 2022-02-02  38743.714844  38834.617188  36832.730469  36952.984375
2696 2022-02-03  36944.804688  37154.601563  36375.539063  37154.601563
2697 2022-02-04  37149.265625  41527.785156  37093.628906  41500.875000
2698 2022-02-05  41501.480469  41847.164063  41038.097656  41441.164063
2699 2022-02-06  41441.121094  42500.785156  41244.906250  42412.433594
2700 2022-02-07  42406.781250  44401.863281  41748.156250  43840.285156
2701 2022-02-08  43854.652344  45293.867188  42807.835938  44118.445313
2702 2022-02-09  44096.703125  44727.800781  43232.968750  44338.796875
2703 2022-02-10  44347.800781  45661.171875  43402.808594  43565.113281
2704 2022-02-11  43571.128906  43810.832031  42114.539063  42407.937500
2705 2022-02-12  42412.300781  42992.550781  41852.574219  42244.468750
2706 2022-02-13  42236.566406  42693.054688  41950.941406  42197.515625
2707 2022-02-14  42157.398438  42775.777344  41681.957031  42586.917969
2708 2022-02-15  42586.464844  44667.218750  42491.035156  44575.203125
2709 2022-02-16  44578.277344  44578.277344  43456.691406  43961.859375
2710 2022-02-17  43937.070313  44132.972656  40249.371094  40538.011719
2711 2022-02-18  40552.132813  40929.152344  39637.617188  40030.976563
2712 2022-02-19  40022.132813  40246.027344  40010.867188  40126.429688

In [55]:
monthvise= y_2022.groupby(y_2022['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January    41368.073463  41114.422379
February   41632.713816  41723.015420
March               NaN           NaN
April               NaN           NaN
May                 NaN           NaN
June                NaN           NaN
July                NaN           NaN
August              NaN           NaN
September           NaN           NaN
October             NaN           NaN
November            NaN           NaN
December            NaN           NaN

In [56]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Open'],
    name='Stock Open Price',
    marker_color='crimson'
))
fig.add_trace(go.Bar(
    x=monthvise.index,
    y=monthvise['Close'],
    name='Stock Close Price',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Monthwise comparision between Stock open and close price')
fig.show()

In [57]:
y_2022.groupby(y_2022['Date'].dt.strftime('%B'))['Low'].min()
monthvise_high = y_2022.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
monthvise_high = monthvise_high.reindex(new_order, axis=0)

monthvise_low = y_2022.groupby(y_2022['Date'].dt.strftime('%B'))['Low'].min()
monthvise_low = monthvise_low.reindex(new_order, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthvise_high.index,
    y=monthvise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthvise_low.index,
    y=monthvise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [58]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_2022, x=y_2022.Date, y=[y_2022['Open'], y_2022['Close'], 
                                          y_2022['High'], y_2022['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# Overall Analysis from 2014-2022

In [59]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')

y_overall = maindf.loc[(maindf['Date'] >= '2014-09-17')
                     & (maindf['Date'] <= '2022-12-31')]

y_overall.drop(y_overall[['Adj Close','Volume']],axis=1)

Date          Open          High           Low         Close
0    2014-09-17    465.864014    468.174011    452.421997    457.334015
1    2014-09-18    456.859985    456.859985    413.104004    424.440002
2    2014-09-19    424.102997    427.834991    384.532013    394.795990
3    2014-09-20    394.673004    423.295990    389.882996    408.903992
4    2014-09-21    408.084991    412.425995    393.181000    398.821014
...         ...           ...           ...           ...           ...
2708 2022-02-15  42586.464844  44667.218750  42491.035156  44575.203125
2709 2022-02-16  44578.277344  44578.277344  43456.691406  43961.859375
2710 2022-02-17  43937.070313  44132.972656  40249.371094  40538.011719
2711 2022-02-18  40552.132813  40929.152344  39637.617188  40030.976563
2712 2022-02-19  40022.132813  40246.027344  40010.867188  40126.429688

[2713 rows x 5 columns]

In [60]:
monthvise= y_overall.groupby(y_overall['Date'].dt.strftime('%B'))[['Open','Close']].mean()
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
monthvise = monthvise.reindex(new_order, axis=0)
monthvise

Open         Close
Date                                 
January    12855.131425  12828.374881
February   12773.077824  12837.802432
March      10918.895761  10957.226324
April      11338.448900  11359.962198
May        10659.455257  10580.209317
June        9299.305977   9294.420703
July        9285.402500   9330.128271
August     11312.971706  11345.157739
September  10489.365578  10462.378150
October    11321.578327  11416.077925
November   12542.362183  12537.441752
December   12391.975010  12391.988926

In [61]:
names = cycle(['Stock Open Price','Stock Close Price','Stock High Price','Stock Low Price'])

fig = px.line(y_overall, x=y_overall.Date, y=[y_overall['Open'], y_overall['Close'], 
                                          y_overall['High'], y_overall['Low']],
             labels={'Date': 'Date','value':'Stock value'})
fig.update_layout(title_text='Stock analysis chart', font_size=15, font_color='black',legend_title_text='Stock Parameters')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 5. Building LSTM Model

* ## First Step is Preparing Data for Training and Testing

* ## Here we are just considering 1 year data for training data 

* ## Since Bitcoin price has drastically flucated from 200 dollar in year 2014 to 15000 dollar in year 2018 to 3000 dollar in year 2019(theses values are apporx) so we will just consider 1 Year to avoid this type of flucation in the data.

* ## As we want to predict Close Price of the Bitcoin so we are just Considering Close aand Date  

In [62]:
# Lets First Take all the Close Price 
closedf = maindf[['Date','Close']]
print("Shape of close dataframe:", closedf.shape)

Shape of close dataframe: (2713, 2)


In [63]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Whole period of timeframe of Bitcoin close price 2014-2022', plot_bgcolor='white', 
                  font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

### Now we will Take data of just 1 Year

In [64]:
closedf = closedf[closedf['Date'] > '2021-02-19']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

Total data for prediction:  365


In [65]:
closedf

Date         Close
2348 2021-02-20  56099.519531
2349 2021-02-21  57539.945313
2350 2021-02-22  54207.320313
2351 2021-02-23  48824.425781
2352 2021-02-24  49705.332031
...         ...           ...
2708 2022-02-15  44575.203125
2709 2022-02-16  43961.859375
2710 2022-02-17  40538.011719
2711 2022-02-18  40030.976563
2712 2022-02-19  40126.429688

[365 rows x 2 columns]

In [66]:
fig = px.line(closedf, x=closedf.Date, y=closedf.Close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered period to predict Bitcoin close price', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

* ### Normalizing Data
- Normalization is a technique often applied as part of data preparation for machine learning. The goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information.
- MinMaxScaler. For each value in a feature, MinMaxScaler subtracts the minimum value in the feature and then divides by the range. The range is the difference between the original maximum and original minimum. MinMaxScaler preserves the shape of the original distribution.

In [67]:
# deleting date column and normalizing using MinMax Scaler

del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

(365, 1)


* ### Slicing data into Training set and Testing set

In [68]:
# we keep the training set as 60% and 40% testing set

training_size=int(len(closedf)*0.80)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

train_data:  (292, 1)
test_data:  (73, 1)


* ### Now we Transform the Close price based on Time-series-analysis forecasting requirement , Here we will take 15   

In [69]:
# convert an array of values into a dataset matrix

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [70]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  (276, 15)
y_train:  (276,)
X_test:  (57, 15)
y_test (57,)


In [71]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (276, 15, 1)
X_test:  (57, 15, 1)


* # Actuall Model Building

In [72]:
model=Sequential()

model.add(LSTM(10,input_shape=(None,1),activation="relu"))

model.add(Dense(1))

model.compile(loss="mean_squared_error",optimizer="adam")

In [73]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

Epoch 1/200
9/9 [==============================] - 6s 105ms/step - loss: 0.3161 - val_loss: 0.1008
Epoch 2/200
9/9 [==============================] - 1s 62ms/step - loss: 0.2547 - val_loss: 0.0727
Epoch 3/200
9/9 [==============================] - 1s 61ms/step - loss: 0.1966 - val_loss: 0.0474
Epoch 4/200
9/9 [==============================] - 0s 45ms/step - loss: 0.1401 - val_loss: 0.0256
Epoch 5/200
9/9 [==============================] - 1s 62ms/step - loss: 0.0865 - val_loss: 0.0101
Epoch 6/200
9/9 [==============================] - 1s 60ms/step - loss: 0.0413 - val_loss: 0.0048
Epoch 7/200
9/9 [==============================] - 1s 63ms/step - loss: 0.0156 - val_loss: 0.0084
Epoch 8/200
9/9 [==============================] - 1s 61ms/step - loss: 0.0100 - val_loss: 0.0056
Epoch 9/200
9/9 [==============================] - 1s 84ms/step - loss: 0.0076 - val_loss: 0.0032
Epoch 10/200
9/9 [==============================] - 1s 69ms/step - loss: 0.0067 - val_loss: 0.0033
Epoch 11/200
9/9 [

- ### Plotting Loss vs Validation loss 

In [74]:
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

<Figure size 432x288 with 0 Axes>

In [75]:
### Lets Do the prediction and check performance metrics
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

2/2 [==============================] - 0s 7ms/step


((276, 1), (57, 1))

- # Model Evaluation

In [76]:
# Transform back to original form

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

- ## Evaluation metrices RMSE, MSE and MAE

In [77]:
# Evaluation metrices RMSE and MAE
print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

Train data RMSE:  1886.3696313152418
Train data MSE:  3558390.3859484014
Train data MAE:  1444.624660355978
-------------------------------------------------------------------------------------
Test data RMSE:  1449.030399450175
Test data MSE:  2099689.098530734
Test data MAE:  1035.3994654736841


- ## Variance Regression Score

In [78]:
print("Train data explained variance regression score:", 
      explained_variance_score(original_ytrain, train_predict))
print("Test data explained variance regression score:", 
      explained_variance_score(original_ytest, test_predict))

Train data explained variance regression score: 0.9653378519005832
Test data explained variance regression score: 0.8644171973115699


- ## R square score for regression

In [79]:
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

Train data R2 score: 0.9647093020823774
Test data R2 score: 0.8623843256138994


- ## Regression Loss Mean Gamma deviance regression loss (MGD) and Mean Poisson deviance regression loss (MPD)

In [80]:
print("Train data MGD: ", mean_gamma_deviance(original_ytrain, train_predict))
print("Test data MGD: ", mean_gamma_deviance(original_ytest, test_predict))
print("----------------------------------------------------------------------")
print("Train data MPD: ", mean_poisson_deviance(original_ytrain, train_predict))
print("Test data MPD: ", mean_poisson_deviance(original_ytest, test_predict))

Train data MGD:  0.0016016702580639352
Test data MGD:  0.0012248870293670468
----------------------------------------------------------------------
Train data MPD:  73.99412681260296
Test data MPD:  50.51043010618397


- # Comparision of original stock close price and predicted close price

In [81]:
# shift train predictions for plotting

look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (365, 1)
Test predicted data:  (365, 1)


- # Predicting next 30 days

In [82]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 30
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  30


- # Plotting last 15 days of dataset and next predicted 30 days

In [83]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39
 40 41 42 43 44 45]


In [84]:
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 30 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 30 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()



- # Plotting entire Closing Stock Price with next 30 days period of prediction

In [85]:
lstmdf=closedf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Close price'])

fig = px.line(lstmdf,labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Plotting whole closing stock price with prediction',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Stock')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

# Thats it we are Done with Bitcoin Price Prediction using LSTM.

![](https://c.tenor.com/nhmb_FF2W1IAAAAM/yaya-for-me-bye.gif)